# Used Car Price Prediction

[Kaggle Resource](https://www.kaggle.com/datasets/taeefnajib/used-car-price-prediction-dataset)

After countless total hours... probably even total days of learning, studying, preparation, examination of best practices, and relearning... it's finally time to get down to business.

I've also recently decided that you can only do so much preparation for a complicated field like ML, so I'm going to likely have to fill in some gaps along the way here. Thankfully, Kaggle has a LOT of publicly posted code to assist me here. I will abstain from referencing it as long as I can though, and will make sure to give credit where it's due.

Anyway, I believe this data marks a perfect balance between a challenging and approachable project. Let's begin.

# Load and Examine Data

In [358]:
import pandas as pd
import re
import numpy as np
import sklearn as skl
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [359]:
cars = pd.read_csv("data/used_cars.csv")
print(len(cars))

4009


Let's just get a quick overview of some basic precleaning stuff.

In [360]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


In [361]:
print(f"Missing Values:\n{cars.isna().sum()}\n\nTotal Duplicates:\n{cars.duplicated().sum()}")

Missing Values:
brand             0
model             0
model_year        0
milage            0
fuel_type       170
engine            0
transmission      0
ext_col           0
int_col           0
accident        113
clean_title     596
price             0
dtype: int64

Total Duplicates:
0


Now that that's out of the way, it's time to actually take a peak at the data.

In [362]:
cars.head(10)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"
5,Acura,ILX 2.4L,2016,"136,397 mi.",Gasoline,2.4 Liter,F,Silver,Ebony.,None reported,NaN,"$14,798"
6,Audi,S3 2.0T Premium Plus,2017,"84,000 mi.",Gasoline,292.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,Blue,Black,None reported,Yes,"$31,000"
7,BMW,740 iL,2001,"242,000 mi.",Gasoline,282.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,A/T,Green,Green,None reported,Yes,"$7,300"
8,Lexus,RC 350 F Sport,2021,"23,436 mi.",Gasoline,311.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,6-Speed A/T,Black,Black,None reported,Yes,"$41,927"
9,Tesla,Model X Long Range Plus,2020,"34,000 mi.",NaN,534.0HP Electric Motor Electric Fuel System,A/T,Black,Black,None reported,Yes,"$69,950"


# Clean the Data

Now that the preliminary stuff is out of the way, let's start cleaning the data up.

It'd be best to start simple here and worry about the basic numeric conversion first. We'll start by removing unnecessary string values from columns that will be designated as strictly numeric. I also think it'd be interesting to see if there are any differences in the model outputs when using `float32` instead of `float64`. We'll get to that much later though.

For now, let's start with simple data cleaning. Then we can chip away at the more tricky regex stuff and increase the complexity as we get a feel for the data.

**Note: We can definitely work on making this SIGNIFICANTLY cleaner by aggregating a lot of this into less lines of code. We want BEST PRACTICES AND THIS IS NOT THAT.**

In [363]:
# Remove non integer string values
cars[["milage", "price"]] = cars[["milage", "price"]].replace(r'[^\d]', '', regex=True)    

# `engine` column
cars["horsepower"] = cars["engine"].str.extract(r'([\d.]+)HP', expand=False)
cars["liters"] = cars["engine"].str.extract(r'([\d.]+)L', expand=False)

# `fuel_type` column
cars["gas_diesel_other"] = (
    cars["fuel_type"].str.contains("gasoline", case=False, na=False) |
    cars["fuel_type"].str.contains("diesel", case=False, na=False) |
    cars["fuel_type"].str.contains("fuel", case=False, na=False)
)
cars["hybrid"] = cars["fuel_type"].str.contains("hybrid", case=False, na=False)
cars["electric"] = cars["fuel_type"].isna()
print(f"Conflicting fuel type values? {(cars[['gas_diesel_other', 'hybrid', 'electric']].sum(axis=1) > 1).any()}")

# `transmission` column
cars["automatic_transmission"] = (
    cars["transmission"].str.contains("A/T", case=False) |
    cars["transmission"].str.contains("AT", case=False) |
    cars["transmission"].str.contains("auto", case=False) |
    cars["transmission"].str.contains("automatic", case=False) |
    cars["transmission"].str.contains("cvt", case=False)
)
cars['dual_shift_transmission'] = cars['transmission'].str.contains("dual", case=False)
cars["manual_transmission"] = (
    cars['transmission'].str.contains("manual", case=False) |
    cars['transmission'].str.contains("m/t", case=False) |
    cars['transmission'].str.contains("mt", case=False)
)
cars.loc[cars['dual_shift_transmission'], ['automatic_transmission', 'manual_transmission']] = False
cars.loc[cars['manual_transmission'], ['automatic_transmission', 'dual_shift_transmission']] = False
mask = ~(cars[['automatic_transmission', 'dual_shift_transmission', 'manual_transmission']] == False).all(axis=1) # Remove rows with `False` in all three newly created transmission columns
cars = cars[mask]
print(f"Conflicting transmission values? {(cars[['automatic_transmission', 'manual_transmission', "dual_shift_transmission"]].sum(axis=1) > 1).any()}")

# `accident` and `clean_title` columns
cars["accident_bool"] = cars["accident"] != "None reported"
cars.dropna(subset=['accident'], inplace=True)

cars.drop(["fuel_type", "engine", "transmission", "accident", "clean_title", "int_col", "ext_col", "model"], axis=1, inplace=True)
cars

Conflicting fuel type values? False
Conflicting transmission values? False


,brand,model_year,milage,price,horsepower,liters,gas_diesel_other,hybrid,electric,automatic_transmission,dual_shift_transmission,manual_transmission,accident_bool
0,Ford,2013,51000,10300,300.0,3.7,True,False,False,True,False,False,True
1,Hyundai,2021,34742,38005,NaN,3.8,True,False,False,True,False,False,True
2,Lexus,2022,22372,54598,NaN,NaN,True,False,False,True,False,False,False
3,INFINITI,2015,88900,15500,354.0,3.5,False,True,False,True,False,False,False
4,Audi,2021,9835,34999,NaN,2.0,True,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4004,Bentley,2023,714,349950,NaN,6.0,True,False,False,True,False,False,False
4005,Audi,2022,10900,53900,349.0,3.0,True,False,False,False,True,False,False
4006,Porsche,2022,2116,90998,NaN,NaN,False,False,True,True,False,False,False
4007,Ford,2020,33000,62999,450.0,3.5,True,False,False,True,False,False,False


 I feel like I need to come up with some better data validation practices. That's for another time though. We'll see if I really messed up when we take a look at the MSE values.

# One-Hot Encode Categorical Data

I'm putting this as it's own section cause I've done one-hot encoding using `pandas`, but not `sklearn`. So let's see if I can get it done (probably gonna be pretty easy). The only categorical column I've kept is `brand`, since the color categories probably don't provide much insight, while there are too many different models in the `model` column to provide useful comparisons (at least I think). However, I think I might have to put this theory to the test.

How about this? We'll add on more requirement to this project: predict `price` using high and low dimensional data and compare the results. This is in addition to the requirement of comparing the various missing value imputation methods we are about to explore as well.

In [364]:
enc = OneHotEncoder(handle_unknown="ignore")
cars = enc.fit_transform(cars[["brand"]])
enc.categories_

[array(['Acura', 'Alfa', 'Aston', 'Audi', 'BMW', 'Bentley', 'Bugatti',
        'Buick', 'Cadillac', 'Chevrolet', 'Chrysler', 'Dodge', 'FIAT',
        'Ferrari', 'Ford', 'GMC', 'Genesis', 'Honda', 'Hummer', 'Hyundai',
        'INFINITI', 'Jaguar', 'Jeep', 'Karma', 'Kia', 'Lamborghini',
        'Land', 'Lexus', 'Lincoln', 'Lotus', 'Lucid', 'MINI', 'Maserati',
        'Maybach', 'Mazda', 'McLaren', 'Mercedes-Benz', 'Mercury',
        'Mitsubishi', 'Nissan', 'Plymouth', 'Polestar', 'Pontiac',
        'Porsche', 'RAM', 'Rivian', 'Rolls-Royce', 'Saab', 'Saturn',
        'Scion', 'Subaru', 'Suzuki', 'Tesla', 'Toyota', 'Volkswagen',
        'Volvo', 'smart'], dtype=object)]

# Fill in Missing Values

Now that the more simple stuff is out of the way, let's start filling in those missing values. The way I see it we have two options: We can either take the mean values for the relevant column of their respective brand, or run some separate regressions to potentially get some more accurate values to fill in. Let's do both and see the results. The second option sounds a bit more interesting and potentially more useful... but I guess we'll have to see what we end up with first.

**An update from Christian in the future:** It seems like it won't be that simple (as usual).

## Fill Missing Values with Brand Mean

Start simple: group by brand name, find the mean of `horsepower` and `liters` for the given brand, then insert the mean values into NaN cells

In [365]:
# Start by converting relevant columns to float64. We can experiment with float32 later
cars[["horsepower", "liters"]] = cars[["horsepower", "liters"]].astype("float64")
brand_avg_hp = cars.groupby(["brand"])["horsepower"].mean()
brand_avg_liters = cars.groupby(["brand"])["liters"].mean()

print(f"Average Horsepower:\n{brand_avg_hp}\nAverage liters:\n{brand_avg_liters}")

UFuncTypeError: ufunc 'maximum' did not contain a loop with signature matching types (dtype('<U10'), dtype('<U10')) -> None

Aha well would you look at that. I forgot to consider that the horsepower might be missing for entire car brands! Therefore, we can't really estimate using the mean of a given brand. Well... we could do an overall average, but that'd be a pretty garbage way to approach the problem.

I also noticed while looking at the results that Bugatti is a car brand in this dataset. While I'm normally an avid hater of removing outliers, this car is way too rare and expensive to be included in this dataset. The other high end brands might be somewhat outliers as well, but not nearly to the extent of a Bugatti.

Let's outline how many cars from each brand can be found in the data then decide where to go from there. Whatever the results though, we definitely need to take Bugatti out of the data.

In [ ]:
cars["brand"].value_counts().sort_values()

brand
Suzuki             1
Polestar           1
Bugatti            1
Maybach            1
smart              1
Plymouth           1
Mercury            2
Saab               2
Karma              2
Lucid              2
Lotus              3
FIAT               5
Saturn             5
McLaren            6
Scion              6
Aston              8
Rolls-Royce        9
Ferrari           12
Pontiac           15
Rivian            15
Alfa              16
Hummer            16
Genesis           19
Mitsubishi        20
Lamborghini       25
Chrysler          27
Buick             30
Bentley           31
MINI              32
Maserati          33
Volvo             37
Jaguar            46
Lincoln           50
INFINITI          55
Acura             57
Volkswagen        59
Honda             60
Subaru            62
Mazda             62
Hyundai           68
Kia               75
Tesla             87
GMC               87
Dodge             89
RAM               90
Cadillac         104
Nissan           113
Land   

Mmmmmm interesting. Thankfully, I'm a bit of a car guy and can identify which of these brands aren't common enough to be needed in the data. However, I think the electric cars can be made as an exception for this, as we need as much electric car data as possible to account for the price difference in electric vs. gas.

I think we can get rid of `Bugatti` and `smart` (maybe a bit of a personal bias towards my hatred of the `smart` brand here, but all-in-all an uncommon brand with outliers in many categories).

In [ ]:
cars = cars[~cars["brand"].isin(["Bugatti", "smart"])]

## Missing Value Imputation

Now let's get a bit more advanced. I'm in uncharted territory with this kind of stuff, but I have a feeling it might not be too challenging. Just gonna take a lot of background studying. Let's go over some of the options we have.

Here's a [general overview](https://scikit-learn.org/stable/modules/impute.html) for Sklearn native missing value imputation. We'll do a deep dive into each Sklearn option in the `sklearn_missing_val_imputation_methods.ipynb` notebook. After that, we'll do the same with [LightGBM](https://lightgbm.readthedocs.io/en/stable/) in a separate notebook, then I can finally create my own attempt using personally selected features for a custom linear regression model. Each imputation method will be assigned to its own data frame.

Before any of that, let's do a quick double check to see if any additional columns have `NaN` values:

In [ ]:
cols_with_nans = [x for x in cars if cars[x].isnull().sum() > 0]
cars[cols_with_nans].isnull().sum()

horsepower    760
liters        360
dtype: int64

Whew... looks like we got quite a few! Already saw this in Data Wrangler but man do we gotta fill these in. Let's begin with the simple Sklearn methods involving mean, median, and mode. I have some serious issues with these approaches, as they are terrible options in my opinion, but hey we're here to learn.

Beyond the methods I'm about to explore, I can also think of a few other ways we could fill in this data even more reliably, but we're here for practice, not the pursuit of the perfect model. That is for another time. Just leaving this note to keep that train of thought in mind. Anyway, let's do the simple stuff now.

Here's a list that will hold the results of each method:

In [ ]:
imp_datasets = {"mean": None,
                "median": None,
                "most_frequent": None}

## Mean, Median, and Most Frequent Value Imputation

Much to my absolute disgust and destain, I think it's important to compare the performance of these more basic missing value imputations to the more advanced methods. This will act as a sort of bonus for familiarizing myself with `scikit`'s syntax and various options.

Let's get to it.

In [ ]:
imp_target_cols = ['horsepower', 'liters']

imp_mean = SimpleImputer(strategy='mean')
cars_mean = cars.copy()
cars_mean[imp_target_cols] = imp_mean.fit_transform(cars[imp_target_cols])

imp_median = SimpleImputer(strategy='median')
cars_median = cars.copy()
cars_median[imp_target_cols] = imp_median.fit_transform(cars[imp_target_cols])

imp_most_freq = SimpleImputer(strategy='most_frequent')
cars_freq = cars.copy()
cars_freq[imp_target_cols] = imp_most_freq.fit_transform(cars[imp_target_cols])

imp_datasets["mean"] = cars_mean
imp_datasets["median"] = cars_median
imp_datasets["most_frequent"] = cars_freq

Well I honestly made that a bit harder than it should've. Was trying to get a bit too fancy with saving the new data frames and instead opted for the more straightforward solution. Let's move onto the `scikit` multivariate options for missing value imputation now.

## Multivariate Missing Value Imputation

Let's see how tricky this'll be.